In [6]:
import numpy as np
import pandas as pd

# load the sample data to evaluate the model
df = pd.read_csv("../data/dow_jones_stocks/sentiments/dataset_1/many_sentiments/AAPL.csv")

# add new column Open_before which contains the open values of the previous day
df["Open_before"] = df["Open"].shift(1)

# calculate the procentual change of the open value of the current day to the 
# open value of the day before
df["Open_changes"] = (df["Open"] / df["Open_before"]) - 1

# throw out the first line which has NaN as value because of the previous shift of values 
df = df.dropna()
df = df.reset_index(drop=True)
df = df.drop(columns=['Unnamed: 0'])

# resort data frame by start backwards
df = df[::-1]

changes = df["Open_changes"]
sentiments = df["Sentiment"]

X = []
Y = []

# start with 1 because we throw out the first line before due to NaN
# predict price with previous 20 entries
for i in range(0, len(changes)- 20):
    Y.append(changes[i])
    to_add = changes[i+1:i+21].tolist()
    to_add.append(sentiments[i])
    X.append(to_add)

# border to split in test and training data
split_border = int(len(X) * 0.8)

# build test and training data
X_train = np.array(X[:split_border]).reshape(-1, 21, 1)
X_test = np.array(X[split_border:]).reshape(-1, 21, 1)
Y_train = np.array(Y[:split_border])
Y_test = np.array(Y[split_border:])

In [7]:
from keras.models import Sequential
from keras.layers import LSTM, Dense

model = Sequential()
model.add(LSTM(128, input_shape=(21, 1), return_sequences=True))
model.add(LSTM(64, input_shape=(21,1)))

model.add(Dense(64, activation="relu"))
model.add(Dense(1, activation="sigmoid"))

model.compile(optimizer="rmsprop", loss="mse")
model.fit(X_train, Y_train, batch_size=32, epochs=10)

model.save('../models/no_normalization_sentiments.h5')

Using TensorFlow backend.
W1110 08:21:03.792096 11688 deprecation_wrapper.py:119] From C:\Users\Daniel\.conda\envs\python36\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1110 08:21:03.815038 11688 deprecation_wrapper.py:119] From C:\Users\Daniel\.conda\envs\python36\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W1110 08:21:03.820026 11688 deprecation_wrapper.py:119] From C:\Users\Daniel\.conda\envs\python36\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W1110 08:21:04.417427 11688 deprecation_wrapper.py:119] From C:\Users\Daniel\.conda\envs\python36\lib\site-packages\keras\optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W1110 

Epoch 1/10
7795/7795 [==============================] - 12s 2ms/step - loss: 0.0049
Epoch 2/10
7795/7795 [==============================] - 12s 2ms/step - loss: 9.7594e-04
Epoch 3/10
7795/7795 [==============================] - 18s 2ms/step - loss: 9.7598e-04
Epoch 4/10
7795/7795 [==============================] - 21s 3ms/step - loss: 9.7491e-04
Epoch 5/10
7795/7795 [==============================] - 22s 3ms/step - loss: 9.7604e-04
Epoch 6/10
7795/7795 [==============================] - 23s 3ms/step - loss: 9.7513e-04
Epoch 7/10
7795/7795 [==============================] - 22s 3ms/step - loss: 9.7523e-04
Epoch 8/10
7795/7795 [==============================] - 26s 3ms/step - loss: 9.7564e-04
Epoch 9/10
7795/7795 [==============================] - 23s 3ms/step - loss: 9.7553e-04
Epoch 10/10
7795/7795 [==============================] - 21s 3ms/step - loss: 9.7601e-04
